# Lab SQL 9
In this lab we will find the customers who were active in consecutive months of May and June. Follow the steps to complete the analysis.


## Create a table `rentals_may` to store the data from rental table with information for the month of May.

In [5]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

import getpass  # to get the password without showing the input
password = getpass.getpass()

connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)

········


In [8]:
engine.execute('''
CREATE TABLE rentals_may(
  rental_id INT NOT NULL AUTO_INCREMENT,
  rental_date DATETIME NOT NULL,
  inventory_id MEDIUMINT UNSIGNED NOT NULL,
  customer_id SMALLINT UNSIGNED NOT NULL,
  return_date DATETIME DEFAULT NULL,
  staff_id TINYINT UNSIGNED NOT NULL,
  last_update TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
  PRIMARY KEY (rental_id),
  UNIQUE KEY  (rental_date,inventory_id,customer_id),
  KEY idx_fk_inventory_id (inventory_id),
  KEY idx_fk_customer_id (customer_id),
  KEY idx_fk_staff_id (staff_id),
  CONSTRAINT fk_rental_staffff FOREIGN KEY (staff_id) REFERENCES staff (staff_id) ON DELETE RESTRICT ON UPDATE CASCADE,
  CONSTRAINT fk_rental_inventoryyyy FOREIGN KEY (inventory_id) REFERENCES inventory (inventory_id) ON DELETE RESTRICT ON UPDATE CASCADE,
  CONSTRAINT fk_rental_customerrrrr FOREIGN KEY (customer_id) REFERENCES customer (customer_id) ON DELETE RESTRICT ON UPDATE CASCADE
)
''')

## Insert values in the table `rentals_may` using the table rental, filtering values only for the month of May.
 

In [10]:
engine.execute('''
insert into sakila.rentals_may 
select * from sakila.rental
where extract(month from rental_date) = '5';
'''
              )

IntegrityError: (pymysql.err.IntegrityError) (1062, "Duplicate entry '1' for key 'rentals_may.PRIMARY'")
[SQL: 
insert into sakila.rentals_may 
select * from sakila.rental
where extract(month from rental_date) = '5';
]
(Background on this error at: http://sqlalche.me/e/13/gkpj)

In [13]:
data = pd.read_sql_query('select * from rentals_may', engine)
data.head()

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53
2,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39,1,2006-02-15 21:30:53
3,4,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41,2,2006-02-15 21:30:53
4,5,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21,1,2006-02-15 21:30:53


## Create a table `rentals_june` to store the data from rental table with information for the month of June.


In [15]:
engine.execute('''
CREATE TABLE rentals_june(
  rental_id INT NOT NULL AUTO_INCREMENT,
  rental_date DATETIME NOT NULL,
  inventory_id MEDIUMINT UNSIGNED NOT NULL,
  customer_id SMALLINT UNSIGNED NOT NULL,
  return_date DATETIME DEFAULT NULL,
  staff_id TINYINT UNSIGNED NOT NULL,
  last_update TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
  PRIMARY KEY (rental_id),
  UNIQUE KEY  (rental_date,inventory_id,customer_id),
  KEY idx_fk_inventory_id (inventory_id),
  KEY idx_fk_customer_id (customer_id),
  KEY idx_fk_staff_id (staff_id),
  CONSTRAINT fk_rental_stafff FOREIGN KEY (staff_id) REFERENCES staff (staff_id) ON DELETE RESTRICT ON UPDATE CASCADE,
  CONSTRAINT fk_rental_inventoryy FOREIGN KEY (inventory_id) REFERENCES inventory (inventory_id) ON DELETE RESTRICT ON UPDATE CASCADE,
  CONSTRAINT fk_rental_customerrr FOREIGN KEY (customer_id) REFERENCES customer (customer_id) ON DELETE RESTRICT ON UPDATE CASCADE
)
''')

## Insert values in the table `rentals_june` using the table rental, filtering values only for the month of June.
 

In [17]:
engine.execute('''
insert into sakila.rentals_june
select * from sakila.rental
where extract(month from rental_date) = '6';
'''
              )

IntegrityError: (pymysql.err.IntegrityError) (1062, "Duplicate entry '1158' for key 'rentals_june.PRIMARY'")
[SQL: 
insert into sakila.rentals_june
select * from sakila.rental
where extract(month from rental_date) = '6';
]
(Background on this error at: http://sqlalche.me/e/13/gkpj)

In [18]:
data2 = pd.read_sql_query('select * from rentals_june', engine)
data2.head()

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1158,2005-06-14 22:53:33,1632,416,2005-06-18 21:37:33,2,2006-02-15 21:30:53
1,1159,2005-06-14 22:55:13,4395,516,2005-06-17 02:11:13,1,2006-02-15 21:30:53
2,1160,2005-06-14 23:00:34,2795,239,2005-06-18 01:58:34,2,2006-02-15 21:30:53
3,1161,2005-06-14 23:07:08,1690,285,2005-06-21 17:12:08,1,2006-02-15 21:30:53
4,1162,2005-06-14 23:09:38,987,310,2005-06-23 22:00:38,1,2006-02-15 21:30:53


## Check the number of rentals for each customer for May.


In [26]:
may_GB = pd.read_sql_query('''
select customer_id, count(rental_id) as `nb_rentals`
from sakila.rentals_may
group by customer_id
order by nb_rentals desc
''', engine
)
may_GB

,customer_id,nb_rentals
0,197,8
1,109,7
2,506,7
3,19,6
4,53,6
...,...,...
515,580,1
516,582,1
517,590,1
518,595,1


## Check the number of rentals for each customer for June.
 

In [27]:
june_GB = pd.read_sql_query('''
select customer_id, count(rental_id) as `nb_rentals`
from sakila.rentals_june
group by customer_id
order by nb_rentals desc
''', engine
)
june_GB

,customer_id,nb_rentals
0,31,11
1,454,10
2,213,9
3,267,9
4,295,9
...,...,...
585,549,1
586,555,1
587,564,1
588,580,1


## Create a Python connection with SQL database and retrieve the results of the last two queries (also mentioned below) as dataframes:

  - Check the number of rentals for each customer for May
  - Check the number of rentals for each customer for June

    **Hint**: You can store the results from the two queries in two separate dataframes.
 

In [30]:
data_1 = pd.DataFrame(may_GB)
data_1

,customer_id,nb_rentals
0,197,8
1,109,7
2,506,7
3,19,6
4,53,6
...,...,...
515,580,1
516,582,1
517,590,1
518,595,1


In [31]:
data_2 = pd.DataFrame(june_GB)
data_2

,customer_id,nb_rentals
0,31,11
1,454,10
2,213,9
3,267,9
4,295,9
...,...,...
585,549,1
586,555,1
587,564,1
588,580,1


## Write a function that checks if customer borrowed more or less films in the month of June as compared to May.
 
 **Hint**: For this part, you can create a join between the two dataframes created before, using the merge function available for pandas dataframes. Here is a link to the documentation for the [merge function](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html).

In [35]:
data_3 = pd.merge(data_1,data_2, how='outer',on='customer_id')
data_3

,customer_id,nb_rentals_x,nb_rentals_y
0,197,8.0,8.0
1,109,7.0,5.0
2,506,7.0,5.0
3,19,6.0,3.0
4,53,6.0,5.0
...,...,...,...
593,335,NaN,1.0
594,370,NaN,1.0
595,487,NaN,1.0
596,555,NaN,1.0


In [41]:
for row in data_3.rows:
    if nb_rentals_x > nb_rentals_y: 
        "Customer borrowed more in May than in June"
    elif nb_rentals_x < nb_rentals_y: 
        "Customer borrowed more in June than in May"
    else "Customer borrowed equally in May and in June"


SyntaxError: invalid syntax (<ipython-input-41-2cfaab7c8f2b>, line 6)